In [1]:
import geopandas as gpd
import pandas as pd
import os
import getpass
import matplotlib.pyplot as plt
import re
import ipyleaflet as ipy 
import json
import ipywidgets

In [2]:
wd = os.getcwd()
wd = wd[:wd.find('notebooks')]

In [3]:
#landuse = gpd.read_file(wd+\"data//")
acl = gpd.read_file(wd+"data/fulljurisdiction/austincitylimitsgj.geojson")
cbg = gpd.read_file(wd+"data/censusblockgroups/cbggj.geojson")
pop = gpd.read_file(wd+"data/blockgroups_censusdata/popcount.geojson")
race = gpd.read_file(wd+"data/blockgroups_censusdata/race.geojson")
income = gpd.read_file(wd+"data/blockgroups_censusdata/income.geojson")
halfbuff = gpd.read_file(wd+"data/coaparks_buffer/halfbuffgj.geojson")
quarbuff = gpd.read_file(wd+"data/coaparks_buffer/quarterbuffgj.geojson")
varibuff = gpd.read_file(wd+"data/coaparks_buffer/varibuffgj.geojson")
parks = gpd.read_file(wd+"data/coaparks/parkboundaries.geojson")
#parks = gpd.read_file("https://data.austintexas.gov/api/geospatial/8f2b-a4q5?method=export&format=GeoJSON")

In [4]:
# Estimate = Total population
# Estimate_1  = Not hispanic or latino
# Estimate_2 = white
# Estimate_3 = Black or African American
# Estimate_4 = American Indian or Native Alaskan
# Estimate_5 = Asian
# esitamte_6 = Native Hawaiian and Other Pasific Islander
# Estimate_7 = Other race
# Estiamte_8 = Non Hispanic - 2 or more Races
# Estimate_9 = Hispanic Or Latino
# Sum of Non is same as total poputaion is the sum of non hispanic and hispanic. 

In [5]:
quarbuf_data = ipy.GeoData(geo_dataframe = quarbuff,
                       style={'color': 'black', 'fillColor': '#eb0c0c', 'opacity':0.5, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Quarter Mile Buffer')

halfbuf_data = ipy.GeoData(geo_dataframe = halfbuff,
                       style={'color': 'black', 'fillColor': '#ffaa00', 'opacity':0.5, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Half Mile Buffer')

varibuf_data = ipy.GeoData(geo_dataframe = varibuff,
                       style={'color': 'black', 'fillColor': '#fcf800', 'opacity':0.5, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Variable Mile Buffer')

pop_data = ipy.GeoData(geo_dataframe=pop,
                    style={'color': 'black', 'fillColor': '#ac7c72', 'opacity':0.5, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                    name = 'Population Data')

#acl_data = ipy.GeoData(geo_dataframe = acl,
                      # style={'color': 'white', 'opacity':0.05, 'weight':5, 'dashArray':'2', 'fillOpacity':0.6},
                       #name = 'Austin City Limits')
#m.add_layer(acl_data)

In [6]:
m = ipy.Map(center=(30.26,-97.74), zoom = 10, basemap= ipy.basemaps.Esri.WorldTopoMap)

def hover_handler(event=None, id=None, properties=None, feature=None):
    label_name.value='Park: '+ str(properties['LOCATION_N'])
    label_parks.value = 'Park acreage: ' + str(properties['ASSET_SIZE'])
    label_buff.value = 'Area within park: '+ str(dropdown.value)

# Not in use anymore..pending deletion
def hover_handler2(event=None, id=None, properties=None, feature=None):
    label_pop.value='Pop: '+ str(properties['Estimate;'])    

def on_dropdown_change(change):
    if change.new == 'Quarter Mile Buffer':
        if len(layer_group.layers) > 2:
            layer_group.remove_layer(layer_group.layers[2])
        layer_group.add_layer(quarbuf_data)
    if change.new == 'Half Mile Buffer':
        if len(layer_group.layers) > 2:
            layer_group.remove_layer(layer_group.layers[2])
        layer_group.add_layer(halfbuf_data)
    if change.new == 'Varied Buffers':
        if len(layer_group.layers) > 2:
            layer_group.remove_layer(layer_group.layers[2])
        layer_group.add_layer(varibuf_data)

        

park_data = ipy.GeoData(geo_dataframe = parks,
                       style={'color': 'black', 'fillColor': '#5c8000', 'opacity':1, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Parks'
                       )


label_parks = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))
label_name = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))
label_buff = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))
label_pop = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))

dropdown = ipywidgets.Dropdown(options=['Quarter Mile Buffer', 'Half Mile Buffer', 'Varied Buffers'],
                    value='Quarter Mile Buffer', 
                    description='Radius within Parks',
                    disabled=False)
display(dropdown)
dropdown.observe(on_dropdown_change)

park_data.on_hover(hover_handler)
pop_data.on_hover(hover_handler2)

#acl_data.on_hover(hover_handler_acl)
#m.add_layer(acl_data)
layer_group = ipy.LayerGroup(layers=(park_data, pop_data, quarbuf_data))

m.add_layer(layer_group)
m.add_control(ipy.LayersControl())
m
ipywidgets.VBox([m, label_name, label_parks, label_buff, label_pop])

Dropdown(description='Radius within Parks', options=('Quarter Mile Buffer', 'Half Mile Buffer', 'Varied Buffer…